In [ ]:
!pip install together
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.environ["TOGETHER_API_KEY"] = ""#INSERT API KEY HERE

import pandas as pd
from datasets import load_dataset
import re
import numpy as np
import torch
from together import Together

In [ ]:
#Loads in the dataset from Hugging face
dataset = load_dataset("csv", data_files="https://huggingface.co/datasets/vkpriya/GPT-WritingPrompts/resolve/main/gpt-writing-prompts.csv.gzip")
df = pd.DataFrame(dataset["train"])

#Loops through the stories to get one human and one AI per prompt up until we reach 150 stories
new_dataset = []
human_turn = True
second = False #I want the second story just because I didn't like the first
for index, row in df.iterrows():
    if row['writer'] == 'human':
        if human_turn:
            if second:
                new_dataset.append(row)
                human_turn = False
                second = False
            else:
                second = True
    else:
        if not human_turn:
            row['story'] = None
            row['writer'] = 'llama'
            new_dataset.append(row)
            human_turn = True
    if len(new_dataset) == 150:
        break
new_df = pd.DataFrame(new_dataset)
#Drop unnecessary columns
new_df = new_df.drop(columns=['Unnamed: 0'])
print(new_df.head())



In [ ]:
#Print the length of the first and 35th story as a sanity check
print(len(df['story'][1]))
print(len(df['story'][35]))


4568
4245


In [ ]:
#Make llama write the stories for all the AI ones so they are no longer GPT written
client = Together()
model="meta-llama/Llama-3.2-3B-Instruct-Turbo"

for index, row in new_df.iterrows():

    if row['writer'] == 'llama':
        prompt = row['prompt']
        response = client.chat.completions.create(
            model = model,
            #using system to prompt
            messages = [
                #{"role": 'system', 'content': "You are a creative writer who likes unconventional novels."},#You are a creative writer who likes unconventional novels.
                {"role": 'user', 'content': 'Write a roughly 500 word short story based on this given prompt:' + prompt}
            ]
        )
        answer_text = response.choices[0].message.content.strip()
        print(len(answer_text))
        new_df.at[index, 'story'] = answer_text
        print('index is done' + str(index))


human:  4568
2728
index is done34
human:  6567
2799
index is done43
human:  2997
2914
index is done49
human:  856
2569
index is done67
human:  3486
2878
index is done92
human:  4119
2818
index is done101
human:  4687
2838
index is done114
human:  3773
2853
index is done140
human:  2558
3170
index is done146
human:  4195
2967
index is done150
human:  2293
3336
index is done162
human:  5225
3313
index is done178
human:  3547
2887
index is done186
human:  2336
3134
index is done199
human:  6557
2693
index is done205
human:  2002
3154
index is done214
human:  3911
3321
index is done229
human:  7284
2947
index is done233
human:  604
2558
index is done239
human:  9277
2833
index is done260
human:  3371
2707
index is done265
human:  11497
3144
index is done272
human:  5802
2899
index is done277
human:  752
3103
index is done309
human:  1555
2278
index is done322
human:  3862
2880
index is done326
human:  2532
2657
index is done331
human:  11187
3316
index is done363
human:  7384
3282
index is

In [ ]:
print(new_df.head())
#Save the new_df with all the stories
new_df.to_csv('gpt-writing-prompts-llama-prompt-1.csv', index=False)


                                               prompt  \
1   You 've finally managed to discover the secret...   
34  You 've finally managed to discover the secret...   
37  The moon is actually a giant egg , and it has ...   
43  The moon is actually a giant egg , and it has ...   
48  For years in your youth the same imaginary cha...   

                                                story writer  
1   I sat nervously in the room , draped almost al...  human  
34  I stared at the business card in my hand, my m...  llama  
37  Sadie 's window was n't very large , no more t...  human  
43  The night sky was ablaze with an otherworldly ...  llama  
48  “ No , no no no ... ” She backed up and turned...  human  
